# Project Overview

Matthew Morgan, B.S

### Introduction

Scraping Twitter for data relating to the field and profession of Genetic Counseling. 
<br>
<br>A common problem in entering the genetic counseling field is lack of exposure and difficulty in finding that exposure to the field of genetic counseling. With many resources being scattered and posted on different channels, it is difficult to locate and combine everything. The goal of this project is to scrape links and information from the internet related to genetic counseling and pool those together as a conglomerate of resources for prospective genetic counseling students to use. This will hopefully provide a more convenient resource for finding exposure and opportunites in genetic counseling.

### Related Works

### Methods

**Focusing on Twitter:**
<br>1: Look for common twitter tags that relate to genetic counseling
    #GCChat
<br>2: Use those tags to find posts that contain links or resources advertisements
<br>3: Analyze tweets for authenticity and usefullness using follower count, length of account, quality of interaction (positive or negative), and more.
<br>4: create a sorted list of resources with descriptions and helpful information
<br>5: (TBD) advertise the list (possibly as a website or a post somewhere? TBD)
<br><br>Started by useing Selenium Chromedriver to gather tweet data, but it soon became apparent that this would be alot of work and would require constant updates as both Chrome and Twitter constantly update HTML format and versions. Next the Tweepy Twitter API was attempted to be used and offered a much simpler way of gathering Twitter data.

### Results

### Discussion